In [26]:
import praw
import requests
import pandas as pd
import json
from datetime import datetime, timedelta, date

from config import reddit_client_id
from config import reddit_secret_key
from config import reddit_user_name
from config import reddit_password
from config import yahoo_api_key

import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.stats import linregress

In [35]:
yahoo_base_url = "https://yahoo-finance127.p.rapidapi.com/price/tsla"
yahoo_base_url = "https://yahoo-finance127.p.rapidapi.com/price/"


headers_yahoo = {
    "X-RapidAPI-Key": '4f21eaecadmshc53af8aa96e4b15p118ecejsnf448733c6bbb',
    "X-RapidAPI-Host": "yahoo-finance127.p.rapidapi.com"
}


#get input from user for stock symbol
company_names = []
stock_symbol = str(input("Enter stock symbol to search"))


#------------------------------------------------------------------------TO-DO - Add error handling for when ticker is not found

# token1 = str(response.json())
#json.dumps(token1, indent=4, sort_keys=True)
yahoo_response = requests.get(yahoo_base_url + stock_symbol, headers=headers_yahoo).json()
#yahoo_response
# print(json.dumps(data, indent=4, sort_keys=True))

#Add longname as search term for reddit
company_names.append(yahoo_response["longName"])

#Add stock symbol as search term for reddit
company_names.append(stock_symbol)

#Get alternate company names to use for reddit search
msg = f"{stock_symbol}'s LongName is {company_names[0]}.  Add another search term and press Enter, or press Cancel when done'"

while True:
    response = str(input(msg))
    if len(response)> 0:
        company_names.append(response)
    else:
        break

#remove duplicates from company_names
reddit_search_list = pd.Series(company_names).drop_duplicates().tolist()

In [28]:
print(yahoo_response)

{'symbol': 'TSLA', 'twoHundredDayAverageChangePercent': {'raw': 0.06561534, 'fmt': '6.56%'}, 'fiftyTwoWeekLowChangePercent': {'raw': 1.3547786, 'fmt': '135.48%'}, 'averageAnalystRating': '2.8 - Hold', 'language': 'en-US', 'regularMarketDayRange': {'raw': '237.4512 - 243.441', 'fmt': '237.45 - 243.44'}, 'earningsTimestampEnd': {'raw': 1706529600, 'fmt': '2024-01-29', 'longFmt': '2024-01-29T07:00'}, 'epsForward': {'raw': 3.56, 'fmt': '3.56'}, 'regularMarketDayHigh': {'raw': 243.441, 'fmt': '243.44'}, 'twoHundredDayAverageChange': {'raw': 14.762009, 'fmt': '14.76'}, 'askSize': {'raw': 11, 'fmt': '11', 'longFmt': '11'}, 'twoHundredDayAverage': {'raw': 224.978, 'fmt': '224.98'}, 'bookValue': {'raw': 16.818, 'fmt': '16.82'}, 'fiftyTwoWeekHighChange': {'raw': -59.550003, 'fmt': '-59.55'}, 'marketCap': {'raw': 762114277376, 'fmt': '762.114B', 'longFmt': '762,114,277,376'}, 'fiftyTwoWeekRange': {'raw': '101.81 - 299.29', 'fmt': '101.81 - 299.29'}, 'fiftyDayAverageChange': {'raw': 5.3636017, 'fm

In [29]:
#Do reddit search

auth = requests.auth.HTTPBasicAuth(reddit_client_id,reddit_secret_key)
data = {
    'grant_type': 'password',
    'username': reddit_user_name,
    'password': reddit_password
}
headers = {'User-Agent': 'redditapi/1'}
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
token12 = res.json()['access_token']
headers = {**headers, **{'Authorization': f'bearer {token12}'}}
headers
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

reddit_posts = []
reddit = praw.Reddit(
    client_id=reddit_client_id,
    client_secret=reddit_secret_key,
    password = data['password'],
    user_agent=f"testscript by u/{data['username']}",
    username = data['username'],
)


#-----------------------------------------------------------TO-DO: Do we need to set a different limit or search criteria?
for term in reddit_search_list:
    for submission in reddit.subreddit("wallstreetbets").search(term, limit=10, sort="new"):
        sub_create_utc = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d') 
        reddit_posts.append({"search term": term,
                             "title": submission.title,
                            "upvotes": submission.score,
                            "num_comments": submission.num_comments,
                            "date_utc": sub_create_utc})
    


reddit_df = pd.DataFrame(reddit_posts)
reddit_df.drop_duplicates(subset=["title"],keep="first",inplace=True)
reddit_df

,search term,title,upvotes,num_comments,date_utc
0,"Tesla, Inc.",JPOW looking at job data today,7,1,2023-12-08
1,"Tesla, Inc.",Where is all the damn gain porn?,36,38,2023-12-07
2,"Tesla, Inc.",TSLA Institutional Ownership Over Time.,11,5,2023-11-28
3,"Tesla, Inc.",Tesla insiders had perfect timing with selling...,94,27,2023-11-20
4,"Tesla, Inc.",$1.2k —> $10.7k thanks to $TSLA,524,69,2023-11-19
5,"Tesla, Inc.",US Senator casually buying leveraged positions...,11260,651,2023-11-17
6,"Tesla, Inc.",I know how much yall like technicals (TSLA),5,22,2023-11-03
7,"Tesla, Inc.","Simple, just swing shares of tsla when RSI hit...",67,28,2023-11-02
8,"Tesla, Inc.",Huge spikes on Tesla chart today - is it anoth...,10,15,2023-10-27
9,"Tesla, Inc.",Tesla's AI can increase its valuation equal of...,21,211,2023-10-27


In [41]:
url = "https://yahoo-finance127.p.rapidapi.com/historic/tsla/1d/100d"

yahoo_response2 = requests.get(url,headers=headers_yahoo).json()
yahoo_response2

{'meta': {'currency': 'USD',
  'symbol': 'TSLA',
  'exchangeName': 'NMS',
  'instrumentType': 'EQUITY',
  'firstTradeDate': 1277818200,
  'regularMarketTime': 1702328401,
  'gmtoffset': -18000,
  'timezone': 'EST',
  'exchangeTimezoneName': 'America/New_York',
  'regularMarketPrice': 239.74,
  'chartPreviousClose': 262.9,
  'priceHint': 2,
  'currentTradingPeriod': {'pre': {'timezone': 'EST',
    'start': 1702285200,
    'end': 1702305000,
    'gmtoffset': -18000},
   'regular': {'timezone': 'EST',
    'start': 1702305000,
    'end': 1702328400,
    'gmtoffset': -18000},
   'post': {'timezone': 'EST',
    'start': 1702328400,
    'end': 1702342800,
    'gmtoffset': -18000}},
  'dataGranularity': '1d',
  'range': '100d',
  'validRanges': ['1d',
   '5d',
   '1mo',
   '3mo',
   '6mo',
   '1y',
   '2y',
   '5y',
   '10y',
   'ytd',
   'max']},
 'timestamp': [1689946200,
  1690205400,
  1690291800,
  1690378200,
  1690464600,
  1690551000,
  1690810200,
  1690896600,
  1690983000,
  1691069

In [42]:
yahoo_finance3 = []

yahoo_finance3.append({"Symbol": yahoo_response2['meta']["symbol"],
                        "Date": yahoo_response2['timestamp'],
                        "Open": yahoo_response2['indicators']['quote'][0]['open'],
                        "Volume": yahoo_response2['indicators']['quote'][0]['volume'],
                        "High": yahoo_response2['indicators']['quote'][0]['high'],
                        "Close": yahoo_response2['indicators']['quote'][0]['close'],
                        "Low": yahoo_response2['indicators']['quote'][0]['low']})

yahoo_finance_df3 = pd.DataFrame(yahoo_finance3)
#yahoo_finance_df3.head()
date_yahoo = yahoo_finance_df3['Date'].tolist()
new_df_yahoo_date = pd.DataFrame(date_yahoo)
#yahoo1 = new_df_yahoo.transpose()
yeah1 = new_df_yahoo_date.astype(int)
yeah1 = yeah1.transpose()
yeah1.columns = ['Date']
date_utc = yeah1['Date'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
yeah1['date_utc'] = date_utc
yeah1

,Date,date_utc
0,1689946200,2023-07-21
1,1690205400,2023-07-24
2,1690291800,2023-07-25
3,1690378200,2023-07-26
4,1690464600,2023-07-27
...,...,...
95,1701786600,2023-12-05
96,1701873000,2023-12-06
97,1701959400,2023-12-07
98,1702045800,2023-12-08


In [43]:

yahoo_finance_df2 = pd.DataFrame(yahoo_finance3)
yahoo_finance_df2.head()

open_15d_yahoo = yahoo_finance_df2['Open'].tolist()
new_df_yahoo = pd.DataFrame(open_15d_yahoo)
#yahoo1 = new_df_yahoo.transpose()
yeahoo2 = new_df_yahoo.astype(int)
yeahoo2 = yeahoo2.transpose()
yeahoo2.columns = ['Open']
yeahoo2

,Open
0,268
1,255
2,272
3,263
4,268
...,...
95,233
96,242
97,241
98,240


In [44]:

yahoo_finance_df2 = pd.DataFrame(yahoo_finance3)
yahoo_finance_df2.head()

open_15d_yahoo2 = yahoo_finance_df2['Close'].tolist()
new_df_yahoo2 = pd.DataFrame(open_15d_yahoo2)
#yahoo1 = new_df_yahoo.transpose()
yeahoo22 = new_df_yahoo2.astype(int)
yeahoo22 = yeahoo22.transpose()
yeahoo22.columns = ['Close']
yeahoo22

,Close
0,260
1,269
2,265
3,264
4,255
...,...
95,238
96,239
97,242
98,243


In [46]:
df100 = pd.concat([yeahoo2['Open'],yeah1['date_utc']], axis='columns')
df200 = pd.concat([yeahoo22['Close'],yeah1['date_utc']], axis='columns')
df100
df200

,Close,date_utc
0,260,2023-07-21
1,269,2023-07-24
2,265,2023-07-25
3,264,2023-07-26
4,255,2023-07-27
...,...,...
95,238,2023-12-05
96,239,2023-12-06
97,242,2023-12-07
98,243,2023-12-08


In [47]:
open_close_df = pd.merge(df100,df200)
open_close_df

,Open,date_utc,Close
0,268,2023-07-21,260
1,255,2023-07-24,269
2,272,2023-07-25,265
3,263,2023-07-26,264
4,268,2023-07-27,255
...,...,...,...
95,233,2023-12-05,238
96,242,2023-12-06,239
97,241,2023-12-07,242
98,240,2023-12-08,243


In [ ]:
merged_df = pd.merge(reddit_df,df100)
merged_df

In [ ]:
# Linear regression on Reddit upvotes and comments
x_values = merged_df['upvotes']
y_values = merged_df['num_comments']

# Perform a linear regression on temperature vs. latitude
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

# Get regression values
regress_values = x_values * slope + intercept
print(regress_values)
print(slope)

line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))
print(line_eq)
# Create Plot
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")

# Label plot and annotate the line equation
plt.xlabel('Open')
plt.ylabel('Comments')
plt.annotate(line_eq,(20,5),fontsize=15,color="red")

# Print r value
print(f"The r-value is: {rvalue**2}")

# Show plot
plt.show()

In [ ]:
from scipy import stats

merged_df = merged_df[(np.abs(stats.zscore(merged_df['upvotes'])) < 3)]
merged_df = merged_df[(np.abs(stats.zscore(merged_df['num_comments'])) < 3)]
merged_df


In [ ]:

yahoo_finance = []

yahoo_finance.append({"Symbol": yahoo_response["symbol"],
                        "Day_Range_raw": yahoo_response["regularMarketDayRange"]["raw"],
                        "Market_Price": yahoo_response["regularMarketPrice"]["raw"],
                        "Company_Name": yahoo_response["longName"]})

yahoo_finance_df = pd.DataFrame(yahoo_finance)
yahoo_finance_df.head()


,Symbol,Day_Range_raw,Market_Price,Company_Name
0,TSLA,237.4512 - 243.441,239.74,"Tesla, Inc."
